REST API 요청 주소
https://developers.kakao.com/docs/latest/ko/local/dev-guide#coord-to-address

api키 확인 주소
https://developers.kakao.com/console/app/826184/config/appKey 

In [2]:
import requests
import pandas as pd
import json
import xml.etree.ElementTree as ET
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()
API_KEY=os.getenv("API_KEY")

KAKAO_API_KEY = os.getenv('KAKAO_API_KEY')
JUSO_API_KEY = os.getenv('JUSO_API_KEY')

In [5]:
#Kakao REST API 

# Kakao API를 사용하여 좌표를 주소로 변환하는 함수
def reverse_geocode(lat, lon):
    api_key = '08fbd2ca704d3731235c44917b68285f'
    url = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"x": lon, "y": lat, "input_coord": "WGS84"}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        result = response.json()
        if result['meta']['total_count'] > 0:
            address = result['documents'][0]
            road_address = address.get('road_address')
            if road_address:
                return road_address['address_name'], address['address']['address_name']
            else:
                return None, address['address']['address_name']
        else:
            return None, "No address found"
    else:
        return None, f"Error: {response.status_code}"


def get_road_address_from_juso(address):
    api_key = 'devU01TX0FVVEgyMDI0MDUyOTIwMTYwODExNDgwNTM='
    api_url = f"http://business.juso.go.kr/addrlink/addrLinkApi.do?currentPage=1&countPerPage=10&keyword={address}&confmKey={JUSO_API_KEY}"
    response = requests.get(api_url)
    
    if response.status_code == 200:
        tree = ET.ElementTree(ET.fromstring(response.content))
        root = tree.getroot()
        road_addr = root.find(".//roadAddrPart1")
        if road_addr is not None:
            return road_addr.text
        else:
            return None
    else:
        return None

# CSV 파일 읽기
df = pd.read_csv('./data/GangnamAdd.csv')

# 결과를 저장할 열 추가
df['road_address'] = None
df['address'] = None

# 각 좌표에 대해 API 호출 및 주소 추가
for idx, row in df.iterrows():
    lat = row['lat']
    lon = row['lng']
    road_addr, addr = reverse_geocode(lat, lon)
    if road_addr is None and addr is not None:
        road_addr = get_road_address_from_juso(addr)
    df.at[idx, 'road_address'] = road_addr
    df.at[idx, 'address'] = addr

# 결과를 새로운 CSV 파일로 저장
df.to_csv('coordinates_with_address.csv', index=False)

KeyboardInterrupt: 

In [47]:
df = pd.read_csv('GangnamAddress.csv')

In [50]:
df[df['road_address'].isna()]

,area,lat,lng,road_address,address
16,17,37.477595,127.056754,None,서울 강남구 압구정동 481
107,108,37.477207,127.057147,None,서울 강남구 압구정동 397
176,177,37.477595,127.056754,None,서울 강남구 일원동 741
190,191,37.477595,127.056754,None,서울 강남구 일원동 741
209,210,37.477595,127.056754,None,서울 강남구 일원동 741
...,...,...,...,...,...
24698,24699,37.494988,127.035638,NaN,서울 강남구 역삼동 751-10
24715,24716,37.499129,127.035362,NaN,서울 강남구 역삼동 736-42
24726,24727,37.499928,127.039716,NaN,서울 강남구 역삼동 725-17
24746,24747,37.498127,127.032210,NaN,서울 강남구 역삼동 823-9


,area,lat,lng,road_address,address
0,1,37.481919,127.057858,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
1,2,37.478823,127.058843,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
2,3,37.480453,127.058339,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
3,4,37.480127,127.063991,서울특별시 강남구 선릉로 8,서울 강남구 개포동 1280
4,5,37.481182,127.057478,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
...,...,...,...,...,...
24762,24763,37.497964,127.039165,서울특별시 강남구 역삼로25길 27,서울 강남구 역삼동 727-2
24763,24764,37.497454,127.033053,None,서울 강남구 역삼동 828-76
24764,24765,37.493709,127.038101,서울특별시 강남구 논현로67길 23,서울 강남구 역삼동 795-2
24765,24766,37.495225,127.032596,서울특별시 강남구 역삼로7길 17,서울 강남구 역삼동 830-41


In [ ]:
# Kakao API를 사용하여 좌표를 주소로 변환하는 함수
def reverse_geocode(lat, lon, api_key):
    url = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"x": lon, "y": lat, "input_coord": "WGS84"}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        result = response.json()
        if result['meta']['total_count'] > 0:
            address = result['documents'][0]
            road_address = address.get('road_address')
            if road_address:
                return road_address['address_name'], address['address']['address_name']
            else:
                return None, address['address']['address_name']
        else:
            return None, "No address found"
    else:
        return None, f"Error: {response.status_code}"

# Kakao REST API 키
api_key = '08fbd2ca704d3731235c44917b68285f'

# 기존 CSV 파일 읽기
df = pd.read_csv('coordinates_with_address1.csv')

# 중복된 좌표를 처리하기 위한 딕셔너리
coordinates_cache = {}

# 도로명 주소가 비어 있는 행들에 대해 API 호출 및 주소 추가
for idx, row in df[df['road_address'].isna()].iterrows():
    lat = row['lat']
    lon = row['lng']
    coords = (lat, lon)
    
    if coords not in coordinates_cache:
        road_addr, addr = reverse_geocode(lat, lon, api_key)
        coordinates_cache[coords] = (road_addr, addr)
    else:
        road_addr, addr = coordinates_cache[coords]
    
    df.at[idx, 'road_address'] = road_addr
    df.at[idx, 'address'] = addr

# 결과를 새로운 CSV 파일로 저장
df.to_csv('coordinates_with_address_updated.csv', index=False)


devU01TX0FVVEgyMDI0MDUyOTIwMTYwODExNDgwNTM=

https://business.juso.go.kr/addrlink/confmKeyRequestSubmit.do


In [5]:
df.to_csv('coordinates_with_address.csv', index=False)

In [14]:
df_part2 = df.iloc[16074:]

In [11]:
df_part1 = df.iloc[:16074]

In [12]:
df_part1

,area,lat,lng,road_address,address
0,1,37.481919,127.057858,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
1,2,37.478823,127.058843,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
2,3,37.480453,127.058339,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
3,4,37.480127,127.063991,서울특별시 강남구 선릉로 8,서울 강남구 개포동 1280
4,5,37.481182,127.057478,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
...,...,...,...,...,...
16069,16070,37.532046,127.033105,서울특별시 강남구 압구정로39길 58,서울 강남구 압구정동 464
16070,16071,37.532497,127.033110,서울특별시 강남구 압구정로39길 58,서울 강남구 압구정동 464
16071,16072,37.532497,127.033110,서울특별시 강남구 압구정로39길 58,서울 강남구 압구정동 464
16072,16073,37.532497,127.033110,서울특별시 강남구 압구정로39길 58,서울 강남구 압구정동 464


In [16]:
df_part2

,area,lat,lng,road_address,address
16074,16075,37.531498,127.032263,None,None
16075,16076,37.531442,127.035330,None,None
16076,16077,37.531921,127.035460,None,None
16077,16078,37.532497,127.033110,None,None
16078,16079,37.531498,127.032263,None,None
...,...,...,...,...,...
24762,24763,37.497964,127.039165,None,None
24763,24764,37.497454,127.033053,None,None
24764,24765,37.493709,127.038101,None,None
24765,24766,37.495225,127.032596,None,None


In [33]:
df = df.iloc[:16074]
df1 = pd.read_csv('coordinates_with_address1.csv')

In [32]:
df

,area,lat,lng,road_address,address
0,1,37.481919,127.057858,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
1,2,37.478823,127.058843,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
2,3,37.480453,127.058339,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
3,4,37.480127,127.063991,서울특별시 강남구 선릉로 8,서울 강남구 개포동 1280
4,5,37.481182,127.057478,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
...,...,...,...,...,...
16069,16070,37.532046,127.033105,서울특별시 강남구 압구정로39길 58,서울 강남구 압구정동 464
16070,16071,37.532497,127.033110,서울특별시 강남구 압구정로39길 58,서울 강남구 압구정동 464
16071,16072,37.532497,127.033110,서울특별시 강남구 압구정로39길 58,서울 강남구 압구정동 464
16072,16073,37.532497,127.033110,서울특별시 강남구 압구정로39길 58,서울 강남구 압구정동 464


In [34]:
df1

,area,lat,lng,road_address,address
0,16075,37.531498,127.032263,서울특별시 강남구 압구정로 201,서울 강남구 압구정동 458
1,16076,37.531442,127.035330,서울특별시 강남구 압구정로 309,서울 강남구 압구정동 481
2,16077,37.531921,127.035460,서울특별시 강남구 압구정로 309,서울 강남구 압구정동 481
3,16078,37.532497,127.033110,서울특별시 강남구 압구정로39길 58,서울 강남구 압구정동 464
4,16079,37.531498,127.032263,서울특별시 강남구 압구정로 201,서울 강남구 압구정동 458
...,...,...,...,...,...
8688,24763,37.497964,127.039165,서울특별시 강남구 역삼로25길 27,서울 강남구 역삼동 727-2
8689,24764,37.497454,127.033053,NaN,서울 강남구 역삼동 828-76
8690,24765,37.493709,127.038101,서울특별시 강남구 논현로67길 23,서울 강남구 역삼동 795-2
8691,24766,37.495225,127.032596,서울특별시 강남구 역삼로7길 17,서울 강남구 역삼동 830-41


In [35]:
df = pd.concat([df, df1])
df

,area,lat,lng,road_address,address
0,1,37.481919,127.057858,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
1,2,37.478823,127.058843,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
2,3,37.480453,127.058339,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
3,4,37.480127,127.063991,서울특별시 강남구 선릉로 8,서울 강남구 개포동 1280
4,5,37.481182,127.057478,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
...,...,...,...,...,...
8688,24763,37.497964,127.039165,서울특별시 강남구 역삼로25길 27,서울 강남구 역삼동 727-2
8689,24764,37.497454,127.033053,NaN,서울 강남구 역삼동 828-76
8690,24765,37.493709,127.038101,서울특별시 강남구 논현로67길 23,서울 강남구 역삼동 795-2
8691,24766,37.495225,127.032596,서울특별시 강남구 역삼로7길 17,서울 강남구 역삼동 830-41


In [36]:
df

,area,lat,lng,road_address,address
0,1,37.481919,127.057858,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
1,2,37.478823,127.058843,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
2,3,37.480453,127.058339,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
3,4,37.480127,127.063991,서울특별시 강남구 선릉로 8,서울 강남구 개포동 1280
4,5,37.481182,127.057478,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
...,...,...,...,...,...
8688,24763,37.497964,127.039165,서울특별시 강남구 역삼로25길 27,서울 강남구 역삼동 727-2
8689,24764,37.497454,127.033053,NaN,서울 강남구 역삼동 828-76
8690,24765,37.493709,127.038101,서울특별시 강남구 논현로67길 23,서울 강남구 역삼동 795-2
8691,24766,37.495225,127.032596,서울특별시 강남구 역삼로7길 17,서울 강남구 역삼동 830-41


In [41]:
df = pd.read_csv('GangnamAddress.csv')

In [43]:
df[df['road_address'].isna()]

,area,lat,lng,road_address,address
294,295,37.477595,127.056754,None,서울 강남구 개포동 1282-5
2079,2080,37.512709,127.039221,None,서울 강남구 논현동 257-19
2437,2438,37.508610,127.033633,None,서울 강남구 논현동 228-8
3936,3937,37.509509,127.035871,None,서울 강남구 논현동 230-12
4192,4193,37.509718,127.033538,None,서울 강남구 논현동 221-7
...,...,...,...,...,...
24698,24699,37.494988,127.035638,None,서울 강남구 역삼동 751-10
24715,24716,37.499129,127.035362,None,서울 강남구 역삼동 736-42
24726,24727,37.499928,127.039716,None,서울 강남구 역삼동 725-17
24746,24747,37.498127,127.032210,None,서울 강남구 역삼동 823-9
